# Trade Idea 


**Long Southern European equities (Spain, Italy, Portugal, Greece) versus Eurozone core equities.**

### Client Pitch

*"We like a relative-value equity trade within Europe. Spain is showing stronger growth momentum, while Italy has suprised on fiscal discipline throughout meaningful government spending cuts. These fundamentals contravt with a slower-moving Eurozone core. We express this by going long Southern Europe equities and short core Eurozone equities over a short tactical horizon."*

### Why now

- Macro narrative is region-specific, not global
- Equity markets tend to reward reltive fiscal and growth improvements
- Clean way to express intra-Europe differentiation without FX risk




## Trade Definition
---

### Macro Hypothesis
- Spain: positive growth impulse
- Italy: fiscal tightening -> credibility improvement
- Southern Europe as a block outperforming Eurozone core

### Instruments

**Long leg (equal-weighted basket)**
- Spain ETF: EWP
- Italy ETF: EWI
- Portugal ETF: PGAL
- Greece ETF: GREK

**Short leg (equal-weighted basket)**
- Germany ETF: EWG
- France ETF: EWQ



## Trade Structure
---

- **Type**: Relative value / regional equity spread
- **Direction**: Long Southern Europe basket, Short Eurozone core basket
- **Holding Period**: 10 trading days
- **Entry**: Today's close
- **Exit**: Time-based: 10 days, Risk -based: stop-loss

## Risk Rules

---

- **Position Sizing**: 1 unit per ETF (equal weight)
- **Stop-Loss**: Trade-level stop at -2%
- No re-entry if stopped
- No parameter optimisation

# Python Implementation

In [22]:
import yfinance as yf
import pandas as pd
import numpy as np

# --- Parameters ---

long_tickers = ["EWP", "EWI", "PGAL", "GREK"]
short_tickers = ["EWG", "EWQ"]

start_date = "2025-01-01"
holding_period = 10
stop_loss = -0.02

# --- Data ---

tickers = long_tickers + short_tickers
prices = yf.download(tickers, start = start_date)['Close']
returns = prices.pct_change().dropna()

# --- Portfolio Construction ---

long_returns = returns[long_tickers].mean(axis = 1)
short_returns = returns[short_tickers].mean(axis = 1)

strategy_returns = long_returns - short_returns
strategy_returns

# --- Trade Simulation ---

trade_pnl = []
trade_dates = []

i = 0
dates = strategy_returns.index

while i < len(strategy_returns) - holding_period:
    cum_return = 0
    stopped = False

    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        cum_return += daily_ret

        if cum_return <= stop_loss:
            trade_pnl.append(cum_return)
            trade_dates.append(dates[i])
            stopped = True
            break

    if not stopped:
        trade_pnl.append(cum_return)
        trade_dates.append(dates[i])
    
    i += holding_period

trade_results = pd.DataFrame({
    "Trade Date": trade_dates,
    "PnL": trade_pnl
})

# --- Performance Summary ---

summary = {
    "Total Trades": len(trade_results),
    "Win Rate": (trade_results["PnL"] > 0).mean(),
    "Average PnL": trade_results["PnL"].mean(),
    "Max Drawdown": trade_results["PnL"].min()
}

print(trade_results.tail())
summary

/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_11562/3110441013.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download(tickers, start = start_date)['Close']
[*********************100%***********************]  6 of 6 completed

   Trade Date       PnL
9  2025-10-21  0.028933
10 2025-11-04  0.021617
11 2025-11-18  0.002717
12 2025-12-03 -0.007664
13 2025-12-17  0.004024



/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_11562/3110441013.py:18: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna()


{'Total Trades': 14,
 'Win Rate': 0.7142857142857143,
 'Average PnL': 0.0069978493311439165,
 'Max Drawdown': -0.015720774390369868}

## Risk & Reality Check

### Key risks
- Trade is equity-beta-exposed despite being relative value
- Short horizon may not allow fiscal story to fully price
- ETF liquidity may demand higher entry price (especially PGAL, GREK)

### Why it could fail
- Risk-off environment -> core Europe outperforms defensively
- Italy fiscal tightening already priced
- US macro dominates European idiosyncrasies

### Volatility intuition
- Medium volatility
- Expect smoother PnL than outright equity trades



## Trader's Journal
---

- **Expected outcome**: Positive
- **Confidence**: 2.6/5
- **Would I put desk capital on this?**: Yes, tactically, with strict stop-loss.

